# Dividends extraction using dedicated service - Python

### Overview
This sample requests **on-demand** dividends by calling a dedicated data service.

### Inputs/outputs
Dividends sample is designed to help you searching and requesting dividends for a given instrument.
* It takes instrument specification parameter(s) as per input filter and returns *Dividends*

### Services used
This sample uses *gRPC requests* in order to retrieve dividends from the hosted service. The queried endpoint in this script are:
* *DividendsService*: to directly retrieve dividends data objects from the server

### Modules required
1. Systemathics packages:
    * *systemathics.apis.services.corporate_actions.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source packages
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    
***

# Run dividends queries

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas

In [ ]:
pip install systemathics.apis --pre

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.services.corporate_actions.v1.dividends_pb2 as dividends
import systemathics.apis.services.corporate_actions.v1.dividends_pb2_grpc as dividends_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Prepare API requests
The following code snippets retrieve authentication token to be used in upcomming API requests:

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Retrieve dividends

#### 3.1 Equity selection

In [ ]:
ticker = 'AAPL'
exchange = 'XNGS'

#### 3.2 Create and process request

In [ ]:
# create the dividends request
request = dividends.DividendsRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker)
)

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        #instantiate the dividends service
        service = dividends_service.DividendsServiceStub(channel)
        
        # process the request
        response = service.Dividends(
            request = request, 
            metadata = [('authorization', token)]
        )
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

#### 3.3 Retrieve dividends

In [ ]:
# prepare the dividends data frame content
dates = [datetime(d.date.year, d.date.month, d.date.day) for d in response.data]
types = [d.type for d in response.data]
amounts = [d.amount for d in response.data]

# Create pandas dataframe
d = {'Date': dates, 'Type': types, 'Amount': amounts}
df = pd.DataFrame(data=d)

In [ ]:
# display dividends
df

#### 3.4 Add dates constraints

In [ ]:
# set data interval (we are using Google date time format) 
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2020, month = 1, day = 1),
    end_date = date.Date(year = 2021, month = 3, day = 31)
)

# generate constraints based on the previous date selection
# time constraints are not neeeded for the dividends
constraints = constraints.Constraints(
    date_intervals = [date_interval]
)

In [ ]:
# create the dividends request with dates constraints
request = dividends.DividendsRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker),
    constraints = constraints
)

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        #instantiate the dividends service
        service = dividends_service.DividendsServiceStub(channel)
        
        # process the request
        response = service.Dividends(
            request = request, 
            metadata = [('authorization', token)]
        )
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

#### 3.5 Retrieve dividends with date constraints

In [ ]:
# prepare the dividends data frame content
dates = [datetime(d.date.year, d.date.month, d.date.day) for d in response.data]
types = [d.type for d in response.data]
amounts = [d.amount for d in response.data]

# Create pandas dataframe
d = {'Date': dates, 'Type': types, 'Amount': amounts}
df = pd.DataFrame(data=d)

In [ ]:
# display dividends
df